
# **Use ANN model to downscale the data, and find their profamnce level**
Use ANN model to downscale the data, and find their profamnce level (2)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.xlsx to Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.xlsx


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
from google.colab import files

# Load historical data
historical_df = pd.read_excel('Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.xlsx')

# Convert month names to numerical values
month_to_num = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
historical_df['Month'] = historical_df['Month'].map(month_to_num)

# Handle missing values in 'Month' column
historical_df.dropna(subset=['Month'], inplace=True)

# Extract predictors and predictands from historical data
X = historical_df[['Latitude', 'Longitude', 'Year', 'Month']].values
y = historical_df[['Tmax', 'Tmin', 'Pr']].values

# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# Define the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = rf_model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the model
joblib.dump(rf_model, 'RF_model_Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.keras')

# Save scalers
joblib.dump(scaler_X, 'Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X.pkl')
joblib.dump(scaler_y, 'Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y.pkl')

print("Random Forest model training complete and saved.")

# Download the model and scaler files
files.download("RF_model_Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.keras")
"""
files.download("Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X.pkl")
files.download("Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y.pkl")
"""

Mean Squared Error: 0.2012665644495871
Random Forest model training complete and saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'\nfiles.download("Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X.pkl")\nfiles.download("Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y.pkl")\n'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X.pkl to Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X (3).pkl
Saving Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y.pkl to Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y (3).pkl
Saving RF_model_Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.pkl to RF_model_Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4 (2).pkl


In [ ]:
import numpy as np
import pandas as pd
import joblib
from google.colab import files

# Load the trained model and scalers with the correct file names
model = joblib.load('RF_model_Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4 (2).pkl')  # Updated to use joblib for scikit-learn model
scaler_X = joblib.load('Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_X (3).pkl')  # Ensure filenames match
scaler_y = joblib.load('Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_scaler_y (3).pkl')

# Load CMIP6 data for predictions
cmip6_df = pd.read_excel('Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4.xlsx')

# Convert month names to numerical values if needed
month_to_num = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
cmip6_df['Month'] = cmip6_df['Month'].map(month_to_num)

# Prepare CMIP6 data for predictions
X_cmip6 = cmip6_df[['Latitude', 'Longitude', 'Year', 'Month']].values
X_cmip6_scaled = scaler_X.transform(X_cmip6)

# Make predictions
y_cmip6_scaled = model.predict(X_cmip6_scaled)

# Inverse transform predictions to get actual values
y_cmip6 = scaler_y.inverse_transform(y_cmip6_scaled)

# Add predictions to the DataFrame
cmip6_df['Predicted_Tmax'] = y_cmip6[:, 0]
cmip6_df['Predicted_Tmin'] = y_cmip6[:, 1]
cmip6_df['Predicted_Pr'] = y_cmip6[:, 2]

# Ensure non-negative precipitation values
cmip6_df['Predicted_Pr'] = np.maximum(cmip6_df['Predicted_Pr'], 0)

# Save the DataFrame with predictions to a CSV file
predictions_file = 'Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_predictions.csv'
cmip6_df.to_csv(predictions_file, index=False)

# Download the file (if using Google Colab)
files.download(predictions_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sorted_Obserevd_data_based_on_requriment.xls to sorted_Obserevd_data_based_on_requriment.xls


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_predictions (1).csv to Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_predictions (1).csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Performance functions
def correlation_coefficient(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]

def nash_sutcliffe_efficiency(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def kling_gupta_efficiency(y_true, y_pred):
    r = np.corrcoef(y_true, y_pred)[0, 1]
    beta = np.mean(y_pred) / np.mean(y_true)
    alpha = np.std(y_pred) / np.std(y_true)
    return 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

def modified_index_of_agreement(y_true, y_pred):
    mean_obs = np.mean(y_true)
    obs_diff = y_true - mean_obs
    pred_diff = y_pred - mean_obs
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((np.abs(pred_diff) + np.abs(obs_diff)) ** 2))

# Load observed data
observed_df = pd.read_excel('sorted_Obserevd_data_based_on_requriment.xls')

# Convert month names to numerical values
month_to_num = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
observed_df['Month'] = observed_df['Month'].map(month_to_num)

# Remove rows with 'ANNUAL' if they exist
observed_df = observed_df.dropna(subset=['Month'])

# Strip trailing spaces and special characters from 'Station Name'
observed_df['Station Name'] = observed_df['Station Name'].str.strip()

# List of prediction files
prediction_files = ['Pr_Tmax_Tmin_ACCES-CM2_SSP-2.4_predictions (1).csv']

# Initialize dictionary to store combined performance metrics
combined_performance_metrics = {
    'Tmax': {'CC': [], 'NSE': [], 'RMSE': [], 'KGE': [], 'md': []},
    'Tmin': {'CC': [], 'NSE': [], 'RMSE': [], 'KGE': [], 'md': []},
    'Precipitation': {'CC': [], 'NSE': [], 'RMSE': [], 'KGE': [], 'md': []}
}

# Process each prediction file
for file in prediction_files:
    # Load predicted data
    predicted_df = pd.read_csv(file)

    # Strip trailing spaces and special characters
    predicted_df['Station Name'] = predicted_df['Station Name'].str.strip()

    # Merge observed and predicted data on common columns
    merged_df = pd.merge(observed_df, predicted_df, on=['Station Name', 'Latitude', 'Longitude', 'Year', 'Month'], how='inner', suffixes=('_obs', '_pred'))

    # Extract relevant columns for performance evaluation
    observed_tmax = merged_df['Observed_Tmax']
    predicted_tmax = merged_df['Tmax']
    observed_tmin = merged_df['Observed_Tmin']
    predicted_tmin = merged_df['Tmin']
    observed_precip = merged_df['Observed_Pr']
    predicted_precip = merged_df['Pr']

    # Calculate performance metrics for Tmax
    combined_performance_metrics['Tmax']['CC'].append(correlation_coefficient(observed_tmax, predicted_tmax))
    combined_performance_metrics['Tmax']['NSE'].append(nash_sutcliffe_efficiency(observed_tmax, predicted_tmax))
    combined_performance_metrics['Tmax']['RMSE'].append(root_mean_squared_error(observed_tmax, predicted_tmax))
    combined_performance_metrics['Tmax']['KGE'].append(kling_gupta_efficiency(observed_tmax, predicted_tmax))
    combined_performance_metrics['Tmax']['md'].append(modified_index_of_agreement(observed_tmax, predicted_tmax))

    # Calculate performance metrics for Tmin
    combined_performance_metrics['Tmin']['CC'].append(correlation_coefficient(observed_tmin, predicted_tmin))
    combined_performance_metrics['Tmin']['NSE'].append(nash_sutcliffe_efficiency(observed_tmin, predicted_tmin))
    combined_performance_metrics['Tmin']['RMSE'].append(root_mean_squared_error(observed_tmin, predicted_tmin))
    combined_performance_metrics['Tmin']['KGE'].append(kling_gupta_efficiency(observed_tmin, predicted_tmin))
    combined_performance_metrics['Tmin']['md'].append(modified_index_of_agreement(observed_tmin, predicted_tmin))

    # Calculate performance metrics for Precipitation
    combined_performance_metrics['Precipitation']['CC'].append(correlation_coefficient(observed_precip, predicted_precip))
    combined_performance_metrics['Precipitation']['NSE'].append(nash_sutcliffe_efficiency(observed_precip, predicted_precip))
    combined_performance_metrics['Precipitation']['RMSE'].append(root_mean_squared_error(observed_precip, predicted_precip))
    combined_performance_metrics['Precipitation']['KGE'].append(kling_gupta_efficiency(observed_precip, predicted_precip))
    combined_performance_metrics['Precipitation']['md'].append(modified_index_of_agreement(observed_precip, predicted_precip))

# Calculate average performance metrics
average_performance_metrics = {}
for key in combined_performance_metrics:
    average_performance_metrics[key] = {metric: np.mean(values) for metric, values in combined_performance_metrics[key].items()}

# Print average performance metrics
print("Average Performance Metrics:")
for category, metrics in average_performance_metrics.items():
    print(f"  {category}:")
    for metric, value in metrics.items():
        print(f"    {metric}: {value:.4f}")

Average Performance Metrics:
  Tmax:
    CC: 0.8645
    NSE: 0.4844
    RMSE: 5.2431
    KGE: 0.6353
    md: 0.9008
  Tmin:
    CC: 0.9063
    NSE: 0.7834
    RMSE: 3.7997
    KGE: 0.8624
    md: 0.9492
  Precipitation:
    CC: 0.1674
    NSE: -0.1162
    RMSE: 71.8610
    KGE: -0.1021
    md: 0.3595
